# Analyzing CIA Factbook Data Using SQL

Database can be downloaded* [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)*

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Taking a look at the facts table

In [3]:
%%sql
SELECT * FROM facts LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


We have columns outlining details on various countries.
Here are the descriptions for some of the columns:

 - name - The name of the country.
 - area - The total land and sea area of the country.
 - population - The country's population.
 - population_growth- The country's population growth as a percentage.
 - birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
 - death_rate - The country's death rate, or the number of death a year per 1,000 people.
 - area- The country's total area (both land and water).
 - area_land - The country's land area in square kilometers.
 - area_water - The country's waterarea in square kilometers.

## Summary statistics

Firstly, we look at both the largest and the smallest populations in the dataset 

In [4]:
%%sql
SELECT MIN(population) 'Minimum population', MAX(population) 'Maximum population', MIN(population_growth) 'Min population growth', MAX(population_growth) 'Max population growth'
  FROM facts;

Done.


Minimum population,Maximum population,Min population growth,Max population growth
0,7256490011,0.0,4.02


In [10]:
%%sql
SELECT name, population FROM facts where population == (SELECT MIN(population) FROM facts);

Done.


name,population
Antarctica,0


Here we see that Antartica has the lowest population with no permanent residents

In [11]:
%%sql
SELECT name, population FROM facts where population == (SELECT MAX(population) FROM facts);

Done.


name,population
World,7256490011


The largest population in the dataset is actually represents the entire world rather any one country. To prevent an overinflation of statistics, we can place an exception for this row in the dataset.  

In [16]:
%%sql
SELECT name, population FROM facts where population = (SELECT MAX(population) FROM facts where name != 'World');

Done.


name,population
China,1367485388


In [17]:
%%sql
SELECT AVG(population), AVG(area) FROM facts where name != 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


Let's take a look at the number of countries with above average populations and below average total areas.

In [18]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population) FROM facts where name != 'World') AND area < (SELECT AVG(area) FROM facts where name != 'World');

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Water to Land Ratio

Here we take a look at the countries with the highest water to land ratios such that they have more water areas than land.

In [24]:
%%sql
SELECT name, area_land, area_water, ROUND((CAST(area_water as float) / CAST(area_land as float)), 2) water_land_ratio
FROM facts
WHERE name != 'World'
ORDER BY water_land_ratio DESC
LIMIT 10;

Done.


name,area_land,area_water,water_land_ratio
British Indian Ocean Territory,60,54340,905.67
Virgin Islands,346,1564,4.52
Puerto Rico,8870,4921,0.55
"Bahamas, The",10010,3870,0.39
Guinea-Bissau,28120,8005,0.28
Malawi,94080,24404,0.26
Netherlands,33893,7650,0.23
Uganda,197100,43938,0.22
Eritrea,101000,16600,0.16
Liberia,96320,15049,0.16


## Population Growth

In [26]:
%%sql
SELECT name, population, population_growth, ROUND((population_growth/100)*population, 2) people_added
FROM facts
where name != 'World'
ORDER BY people_added DESC
LIMIT 10;

Done.


name,population,population_growth,people_added
India,1251695584,1.22,15270686.12
China,1367485388,0.45,6153684.25
Nigeria,181562056,2.45,4448270.37
Pakistan,199085847,1.46,2906653.37
Ethiopia,99465819,2.89,2874562.17
Bangladesh,168957745,1.6,2703323.92
United States,321368864,0.78,2506677.14
Indonesia,255993674,0.92,2355141.8
"Congo, Democratic Republic of the",79375136,2.45,1944690.83
Philippines,100998376,1.61,1626073.85


The table above shows the countries with the highest population growth in terms of the number of people added yearly.

The next query looks at which countries have a higher death rate than birth rate.

In [27]:
%%sql
SELECT name, population, birth_rate, death_rate
FROM facts 
WHERE death_rate > birth_rate
ORDER BY death_rate DESC
LIMIT 10

Done.


name,population,birth_rate,death_rate
Ukraine,44429471,10.72,14.46
Bulgaria,7186893,8.92,14.44
Latvia,1986705,10.0,14.31
Lithuania,2884433,10.1,14.27
Russia,142423773,11.6,13.69
Serbia,7176794,9.08,13.66
Belarus,9589689,10.7,13.36
Hungary,9897541,9.16,12.73
Moldova,3546847,12.0,12.59
Estonia,1265420,10.51,12.4
